In [87]:
from itertools import product
import os
import pandas as pd
import shutil

In [88]:
instance_set = "miplib_2017_5000_v2"
max_runtime = 3600
expected_instances = 4
degrees = [-1, 1]
perturbations = ["matrix", "objective", "rhs"]

In [89]:
# running list of strings contained by different error codes
# last two are catchalls
err = {
    "gurobipy.gurobierror: out of memory": [],
    f"could not be solved in": []
}

warn = {
    "invalid value encountered in double_scalars": [],
    "could not be perturbed": [],
}

# runs that weren't run
no_go = []

# runs that errored out with new error code
other = []

# runs that had no errors
empty = []

# count of instances made
count = {}

for i, file_name in enumerate(os.listdir(os.path.join("instances", instance_set))):
    
    # skip anything not a mip
    if not file_name.endswith(".mps"):
        continue
        
    # get the model name
    stem = file_name[:-4]
    
    # instantiate the count
    count[stem] = {}
    for p, d in product(perturbations, degrees):
        if os.path.isdir(os.path.join("test_sets", instance_set, stem, f"{p}_{d}")):
            count[stem][(p, d)] = len([f for f in os.listdir(os.path.join("test_sets", instance_set, stem, f"{p}_{d}")) if f.endswith(".mps")])
    
    # get the error file path
    err_pth = os.path.join("outfiles", stem + ".err")
    
    # check if the series wasn't run
    if not os.path.exists(err_pth):
        no_go.append(stem)
    
    # check if the series ran with no errors or warnings
    elif os.path.getsize(err_pth) == 0:
        empty.append(stem)
        
    else:
        # read in file
        with open(err_pth, "r") as f:
            text = f.read().lower()
            
        # check for error codes
        found_code = False
        for code in err:
            if code in text:
                err[code].append(file_name)
                found_code = True
                break
        
        if not found_code:
            for code in warn:
                if code in text:
                    warn[code].append(file_name)
                    found_code = True
                    break
                    
        if not found_code:
            other.append(file_name)

In [90]:
no_go

['mkc1',
 'pigeon-13',
 'markshare1',
 'pw-myciel4',
 'bnatt400',
 'usAbbrv-8-25_70',
 'neos-1442119',
 'mitre',
 'gen-ip016']

In [91]:
len(empty) / (i + 1)

0.6420765027322405

In [92]:
bumpable = []
print(err["gurobipy.gurobierror: out of memory"])
print(len(err["gurobipy.gurobierror: out of memory"]) / (i + 1))
df = pd.read_csv("more_memory.csv", index_col=0)
for file_name in err["gurobipy.gurobierror: out of memory"]:
    current_memory = df.loc[file_name, "memory"]
    df.loc[file_name, "memory"] = min(15, current_memory * 2)
    if current_memory < 15:
        bumpable.append(file_name)
print(len(bumpable) / (i + 1))
# df["memory"].astype(int)
# df.to_csv("more_memory.csv")

['neos-5075914-elvire.mps', 'pigeon-16.mps', 'milo-v13-4-3d-4-0.mps', 'neos-3046601-motu.mps', 'radiationm18-12-05.mps', 'neos-5078479-escaut.mps', 'nag.mps', 'neos-3592146-hawea.mps', 'neos-3024952-loue.mps', 'mc11.mps', 'bppc6-06.mps', 'neos-3009394-lami.mps', 'cvrpp-n16k8vrpi.mps', 'neos-5045105-creuse.mps', 'neos-3083819-nubu.mps', 'sct2.mps', 'bppc6-02.mps', 'stoch-vrpvrp-s5v2c8vrp-v2c8i.mps', 'neos-3762025-ognon.mps', 'mkc.mps', 'ger50-17-ptp-pop-6t.mps', 'maxgasflow.mps', 'neos-3046615-murg.mps', 'n6-3.mps', 'dfn-bwin-DBE.mps', 'neos-3426085-ticino.mps', 'gsvm2rl9.mps', 'supportcase20.mps', 'fhnw-binpack4-77.mps', 'breastcancer-regularized.mps', 'ta1-UUM.mps', 'bley_xs1noM.mps', 'd20200.mps', 'pigeon-20.mps', 'neos-3373491-avoca.mps', 'ger50-17-ptp-pop-3t.mps', 'liu.mps', 'probportfolio.mps', 'neos-3068746-nene.mps', 'neos-3211096-shag.mps']
0.1092896174863388
0.0


In [93]:
len(err[f"could not be solved in"]) / (i + 1)

0.15846994535519127

In [94]:
len(warn["invalid value encountered in double_scalars"]) / (i + 1)

0.05737704918032787

In [95]:
len(warn["could not be perturbed"]) / (i + 1)

0.00819672131147541

In [96]:
print(other)
len(other) /( i + 1)

[]


0.0

In [97]:
complete = []
removals = []
for stem in count:
    if len([count for (p, d), count in count[stem].items() if count < expected_instances]) == 0:
        complete.append(stem)
    else:
        # remove the directory
        if os.path.isdir(os.path.join("test_sets", instance_set, stem)):
            # shutil.rmtree(os.path.join("test_sets", instance_set, stem))
            removals.append(stem)
len(complete) / (i + 1)

1.0

In [98]:
len(removals) / (i + 1)

0.0

In [104]:
# delete all instances above expected
expected_combos = [f"_{i}{file_type}" for i in range(expected_instances) for file_type in [".pb", ".mps"]]
for stem in count:
    for (p, d), val in count[stem].items():
        if val > expected_instances and os.path.exists(os.path.join("test_sets", instance_set, stem, f"{p}_{d}")):
            for file_name in os.listdir(os.path.join("test_sets", instance_set, stem, f"{p}_{d}")):
                if not any(file_name.endswith(ending) for ending in expected_combos):
                    # os.remove(os.path.join("test_sets", instance_set, stem, f"{p}_{d}", file_name))
                    print("removed: ", file_name)

removed:  b1c1s1_15.pb
removed:  b1c1s1_11.pb
removed:  b1c1s1_5.pb
removed:  b1c1s1_10.pb
removed:  b1c1s1_4.pb
removed:  b1c1s1_14.pb
removed:  b1c1s1_9.mps
removed:  b1c1s1_8.mps
removed:  b1c1s1_5.mps
removed:  b1c1s1_4.mps
removed:  b1c1s1_9.pb
removed:  b1c1s1_6.mps
removed:  b1c1s1_7.mps
removed:  b1c1s1_8.pb
removed:  b1c1s1_7.pb
removed:  b1c1s1_13.pb
removed:  b1c1s1_16.mps
removed:  b1c1s1_14.mps
removed:  b1c1s1_15.mps
removed:  b1c1s1_11.mps
removed:  b1c1s1_16.pb
removed:  b1c1s1_10.mps
removed:  b1c1s1_12.mps
removed:  b1c1s1_6.pb
removed:  b1c1s1_12.pb
removed:  b1c1s1_13.mps
removed:  b1c1s1_22.mps
removed:  b1c1s1_21.pb
removed:  b1c1s1_15.pb
removed:  b1c1s1_23.mps
removed:  b1c1s1_21.mps
removed:  b1c1s1_11.pb
removed:  b1c1s1_5.pb
removed:  b1c1s1_25.pb
removed:  b1c1s1_20.mps
removed:  b1c1s1_18.mps
removed:  b1c1s1_24.mps
removed:  b1c1s1_30.mps
removed:  b1c1s1_10.pb
removed:  b1c1s1_4.pb
removed:  b1c1s1_24.pb
removed:  b1c1s1_25.mps
removed:  b1c1s1_19.mps
rem